In [103]:
import pandas as pd
import numpy as np
import warnings

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [104]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/svm_classifier.csv')

In [105]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [106]:
### Season to test results

N = 2021

In [107]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.podium.values)

In [108]:
def score_classification(model):
    predictions = []
    prob = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make Predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        predictions += list(prediction_df['predicted'].values)  
        prob += list(prediction_df['proba_1'].values)

    return predictions, prob

In [109]:
parameters

,model,gamma,C,kernel,score
0,svm_classifier,0.023357,10.000000,poly,0.526316
1,svm_classifier,0.033598,3.359818,poly,0.526316
2,svm_classifier,0.048329,1.128838,poly,0.526316
3,svm_classifier,0.069519,0.379269,poly,0.526316
4,svm_classifier,0.100000,0.127427,poly,0.526316


In [110]:
params = parameters.iloc[0]

gamma = params.gamma
c = params.C
kernel = params.kernel

params

model     svm_classifier
gamma           0.023357
C                   10.0
kernel              poly
score           0.526316
Name: 0, dtype: object

In [111]:
model = svm.SVC(probability=True, gamma=gamma, C=c, kernel=kernel)
model.fit(X_train, y_train)

predictions, probs = score_classification(model)

In [118]:
df = data.copy()
df = df[(df.season == N)]

df = df[['season', 'round', 'podium', 'q_delta', 'starting_grid',
       'driver_points_before', 'constructor_points_before',
       'driver_points_from', 'driver_points_per', 'points_percentage', 'driver_last_3', 'constructor_last_3']]

df['predicted'] = predictions
df['proba_1'] = probs


In [121]:
df.query('season == 2021 & round == 4').sort_values('proba_1', ascending=False)

,season,round,podium,q_delta,starting_grid,driver_points_before,constructor_points_before,driver_points_from,driver_points_per,points_percentage,driver_last_3,constructor_last_3,predicted,proba_1
2801,2021,4,1,0.000,1,69.0,101.0,25.0,17.25,68.32,69.0,101.0,1,0.249605
2810,2021,4,10,1.241,12,7.0,9.0,1.0,1.75,77.78,7.0,9.0,0,0.053185
2804,2021,4,4,0.769,4,28.0,42.0,12.0,7.00,66.67,28.0,42.0,0,0.044560
2809,2021,4,9,0.839,5,8.0,13.0,2.0,2.00,61.54,8.0,13.0,0,0.036623
2811,2021,4,11,1.233,11,5.0,5.0,0.0,1.25,100.00,5.0,5.0,0,0.032335
2817,2021,4,17,1.406,10,5.0,13.0,0.0,1.25,38.46,5.0,13.0,0,0.031380
2820,2021,4,20,1.815,16,2.0,9.0,0.0,0.67,22.22,2.0,9.0,0,0.025618
2815,2021,4,15,1.615,14,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0,0.025215
2807,2021,4,7,0.879,6,14.0,42.0,6.0,3.50,33.33,14.0,42.0,0,0.025075
2814,2021,4,14,2.413,15,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0,0.023667
